In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s
import ml_window as m
from icecream import ic

import pickle

import plotly.graph_objects as go

In [2]:
a00, a01 = h.import_desired_data("A", "15T")
window_sched = np.array(a01["Window Open"])

In [3]:
with open('../constants/data_combos_230638.pkl', 'rb') as handle:
    test_dict = pickle.load(handle)

In [4]:
for k, v in test_dict.items():
    # print(k)
    obj = m.ML_Window_Detect(v["data"], window_sched)
    obj.run_all()
    test_dict[k]["accuracy"] = obj.accuracy
    test_dict[k]["choices"] = obj.choices

In [31]:
def get_guess_times(test_dict_item):
    choice_list = pd.Series(test_dict_item["choices"])
    mask = choice_list.shift() != choice_list
    guesses = choice_list[mask].index
    guess_times = a01["DateTime"][guesses]

    return guess_times

for k in test_dict.keys():
    s1 = s.Scores(a01)
    g = get_guess_times(test_dict[k])
    test_dict[k]["custom_report"] = s1.report_scores(g)

# with open("../constants/combo_results_230638.pkl", "wb") as fp:
#     pickle.dump(test_dict, fp )

In [38]:
for k, v in test_dict.items():
    print(k, v["custom_report"].loc["(hits + near hits)/actions"][0])

dt_meas 153.8
dt_sim 69.19999999999999
dt_amb 7.7
dt_meas+dt_sim+dt_amb 92.30000000000001
meas 69.19999999999999
amb 53.800000000000004
sim 53.800000000000004
meas+amb+sim 38.5
dt_meas+dt_sim 107.69999999999999
dt_meas+dt_amb 130.8
dt_meas+meas 53.800000000000004
dt_meas+amb 123.10000000000001
dt_meas+sim 107.69999999999999
dt_sim+dt_amb 100.0
dt_sim+meas 92.30000000000001
dt_sim+amb 76.9
dt_sim+sim 92.30000000000001
dt_amb+meas 107.69999999999999
dt_amb+amb 153.8
dt_amb+sim 146.2
meas+amb 53.800000000000004
meas+sim 61.5
amb+sim 61.5
amb_minus_meas 92.30000000000001
meas_minus_amb 92.30000000000001
abs_amb_meas_diff 100.0


In [44]:
# look at choices(t) and window_open(t), are they the same? 
wS = pd.Series(window_sched)
tS = pd.Series(test_dict["amb+sim"]["choices"])
k = wS.compare(tS)
k

,self,other
0,0.0,1.0
1,0.0,1.0
21,0.0,1.0
22,0.0,1.0
23,0.0,1.0
...,...,...
338,0.0,1.0
339,0.0,1.0
340,0.0,1.0
341,0.0,1.0


In [45]:
len(k)

164